In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
f=open(r'./data/lianjia1.csv',encoding='gbk')
data=pd.read_csv(f)

In [ ]:
data.head()

In [ ]:
data_list1=[]
for i in range(1,8):
    dataPath = './data/lianjia'+str(i)+'.csv'
    f=open(dataPath, encoding='gbk', errors='ignore')
    data=pd.read_csv(f)
    data_list1.append(data)

In [ ]:
data_list=[]
for i in range(1,8):
    try:
        f=open(r'./data/lianjia{}.csv'.format(i),encoding='gbk')
        data=pd.read_csv(f)
    except:
        f=open(r'./data/lianjia{}.csv'.format(i),encoding='utf-8')
        data=pd.read_csv(f)
    data_list.append(data)

In [ ]:
len(data_list)

In [ ]:
data_list[2].tail()

In [ ]:
data=pd.concat(data_list)

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
data=data[['cjdanjia','cjxiaoqu','cjlouceng','bankuai','xingming','cjzongjia','congyenianxian','cjshijian','mendian','zhiwei']]

In [ ]:
data.head(3)

In [ ]:
# 数据预处理之前，考虑这个dataframe有没有null值缺失值，有没有异常值，否则直接用一个函数处理会报错。

In [ ]:
data.isnull()

In [ ]:
(data.isnull()).sum()

In [ ]:
data[data.cjdanjia.isnull()]

In [ ]:
data.dropna(how='all',inplace=True)

In [ ]:
data.isnull().sum()

In [ ]:
(data.duplicated()).sum()

### data.drop_duplicates()  的作用是直接抛弃重复值。
### 但是要考虑到板块的数据有很多缺失值，因为drop_duplicates是默认抛弃第二条重复的值
### 若两条数据，前面的cjdnajia，cjxiaoqu，cilouceng是重复的，但是第一条的bankuai为null值，第二条是有数据的，则会默认抛弃第二条有效信息
### 为了避免这种情况，我们要把null值的那条全部放到后面去。

In [ ]:
(data.duplicated(subset=['cjdanjia','cjxiaoqu','cjlouceng','bankuai'])).sum()

In [ ]:
data.sort_values(by='bankuai',inplace=True)

In [ ]:
data.drop_duplicates(subset=['cjdanjia','cjxiaoqu','cjlouceng'],inplace=True)

In [ ]:
#数据类型转换，数据异常值处理，数据离散化处理

In [ ]:
data.head()

In [ ]:
(~data.cjdanjia.str.contains('元/平')).sum()

In [ ]:
data.cjdanjia.map(lambda x:round(float(x.replace('元/平',''))/10000,2))

In [ ]:
data.cjdanjia.str.replace('元/平','').astype(np.float32).map(lambda x:round(x/10000,2))

In [ ]:
data=data.assign(cjdanjia=data.cjdanjia.str.replace('元/平','').astype(np.float32).map(lambda x:round(x/10000,2)))

In [ ]:
data['cjdanjia']=data.cjdanjia.str.replace('元/平','').astype(np.float32).map(lambda x:round(x/10000,2))

In [ ]:
data.head()

In [ ]:
data.cjdanjia.min()

In [ ]:
data.cjdanjia.max()

In [ ]:
data=data[data.cjdanjia>0]

In [ ]:
data.cjdanjia.min()

In [ ]:
data=data[data.cjdanjia>0.5]

In [ ]:
data.cjdanjia.min()

In [ ]:
len(data)

In [ ]:
bins=[0,1,2,3,4,5,7,9,11,13,15]
pd.cut(data.cjdanjia,bins)

In [ ]:
pd.cut(data.cjdanjia,bins).value_counts()

In [ ]:
pd.cut(data.cjdanjia,bins).value_counts().plot()

In [ ]:
pd.cut(data.cjdanjia,bins).value_counts().plot.bar(rot=20)

In [ ]:
pd.cut(data.cjdanjia,bins).value_counts().plot.pie(figsize=(8,8))

# 字符串处理

In [ ]:
data.head()

In [ ]:
(data.cjlouceng.str.split('/').map(len)!=3).sum()

In [ ]:
data.cjlouceng

In [ ]:
data.cjlouceng.map(lambda x:x.split('/')[0])

In [ ]:
data['chaoxiang']=data.cjlouceng.map(lambda x:x.split('/')[0])

In [ ]:
data

In [ ]:
data['louceng']=data.cjlouceng.map(lambda x:x.split('/')[1])

In [ ]:
data

In [ ]:
data.louceng.unique()

In [ ]:
#还有未知？还有空字符串？对于这部分数据我们可以处理一下。

In [ ]:
data[data.louceng=='']

In [ ]:
data[data.louceng=='未知']

In [ ]:
data=data[(data.louceng!='未知')&(data.louceng!='')]

In [ ]:
data

In [ ]:
pd.get_dummies(data.louceng)

In [ ]:
data.join(pd.get_dummies(data.louceng))

In [ ]:
data

In [ ]:
pd.get_dummies(data.louceng).sum()

In [ ]:
pd.get_dummies(data.louceng).sum().plot.bar()

In [ ]:
(pd.get_dummies(data.louceng).sum()).to_csv('loucengfenbu3.csv',encoding='utf_8_sig')

# 分组运算与布尔过滤与透视表

In [ ]:
data.head()

In [ ]:
data['cjshijian']=data.cjshijian.map(lambda x:x.split('：')[1])

In [ ]:
data['year']=data.cjshijian.map(lambda x:x.split('-')[0])

In [ ]:
data.groupby(['year','xingming'])['xingming'].value_counts()

In [ ]:
data.groupby(['year','xingming'])['xingming'].count()

In [ ]:
data.groupby(['year','xingming'])['xingming'].count().to_csv('butongnianfenjingjiren.csv',encoding='utf_8_sig')

In [ ]:
data.groupby(['year','mendian'])['mendian'].count()

In [ ]:
data_group=data.groupby(['xingming','congyenianxian'])['cjzongjia'].sum()

In [ ]:
data_group[data_group>10000]

In [ ]:
data_group[data_group>10000].plot()

In [ ]:
data_1w=data.pivot_table('cjzongjia',index='xingming',columns='congyenianxian',aggfunc=sum)
data_1w

In [ ]:
(data_1w>10000).sum()

In [ ]:
plt.rcParams['font.sans-serif']=['SimHei']  #用来显示中文标签
(data_1w>10000).sum().plot.bar()

In [ ]:
plt.rcParams['font.sans-serif']=['simhei']  #用来显示中文标签


In [ ]:
(data_1w>10000).sum().plot.bar()

In [ ]:
import matplotlib
matplotlib.matplotlib_fname()

In [ ]:
((data_1w>10000).sum()).to_csv('nianxianfenbu1.csv',encoding='utf_8_sig')

# 自己的分析

In [ ]:
data.head()

In [ ]:
data.head()

In [ ]:
data.groupby('year')['cjdanjia'].count()

In [ ]:
data.groupby('year')['cjdanjia'].count().to_csv('chengjiaoliang_year.csv',encoding='utf_8_sig')

In [ ]:
data.groupby('year')['cjdanjia'].mean()

In [ ]:
(data.groupby('year')['cjdanjia'].mean()).to_csv('danjiajunzhi_year.csv',encoding='utf_8_sig')

In [ ]:
data.groupby('year')['cjzongjia'].mean()

In [ ]:
(data.groupby('year')['cjzongjia'].mean()).to_csv('chengjiaozongjia_year.csv',encoding='utf_8_sig')

In [ ]:
data.groupby('bankuai')['cjdanjia'].count()

In [ ]:
(data.groupby('bankuai')['cjdanjia'].count()).to_csv('butongbankuaichengjiao.csv',encoding='utf_8_sig')

In [ ]:
bins=[0,2,4,6,8,12,15]
fenbu=pd.cut(data['cjdanjia'],bins)

In [ ]:
data['cjdanjiafenbu']=fenbu

In [ ]:
data.groupby(['bankuai','cjdanjiafenbu'])['cjdanjiafenbu'].count()

In [ ]:
(data.groupby(['bankuai','cjdanjiafenbu'])['cjdanjiafenbu'].count()).to_csv('cjdanjiafenbu.csv',encoding='utf_8_sig')

In [ ]:
data.groupby(['year','mendian'])['mendian'].count()

In [ ]:
(data.groupby(['year','mendian'])['mendian'].count()).to_csv('butongnianxianmendian.csv',encoding='utf_8_sig')

In [ ]:
data.groupby('zhiwei')['zhiwei'].count()

In [ ]:
(data.groupby('zhiwei')['zhiwei'].count()).to_csv('zhiwei.csv',encoding='utf_8_sig')

In [ ]:
bins=[0,2,4,6,8,12,15]
fenbu=pd.cut(data['cjdanjia'],bins)
data.groupby(['year','cjdanjiafenbu'])['cjdanjiafenbu'].count()

In [ ]:
(data.groupby(['year','cjdanjiafenbu'])['cjdanjiafenbu'].count()).to_csv('xiaofeizhejiage.csv',encoding='utf_8_sig')